<a href="https://colab.research.google.com/github/mforoozandeh/ctrl-freeq/blob/main/src/examples/api_gpu_cpu_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ctrl-freeq: CPU vs CUDA demo - Google Colab
# This notebook demonstrates selecting compute_resource (cpu/gpu) in the API and comparing runtimes.


Clone the repo
```
git clone https://github.com/mforoozandeh/ctrl-freeq.git
GITHUB_TOKEN=<REDACTED>
```

In [ ]:
import sys
sys.path.append("/content/ctrl-freeq")

In [ ]:
%cd /content/ctrl-freeq

In [ ]:
!pwd

In [ ]:
# Ensure Python 3.13.7 is installed and a virtual environment is created
!uv python install 3.13.7
!uv venv --python 3.13.7 --clear

In [ ]:
# Activate the virtual environment and check the Python version
!source .venv/bin/activate && python --version

In [ ]:
# Activate the virtual environment, install uv within it, and check the Python version and uv version
!source .venv/bin/activate && pip install uv && python --version && uv --version

In [ ]:
# Install dependencies using globally installed uv, specifying the virtual environment's python
!uv pip install -e . --python .venv/bin/python

In [ ]:
!pip install pytorch-minimize

In [ ]:
!pip install qiskit_algorithms

In [ ]:
import time
import torch
from src.ctrl_freeq.api import CtrlFreeQAPI, load_single_qubit_config

print("CUDA available:", torch.cuda.is_available())
compute_resource = "gpu" if torch.cuda.is_available() else "cpu"
print("Requested compute_resource =", compute_resource)

# Load example config via API, then extract a mutable dict
base_api = load_single_qubit_config()
config = dict(base_api.config)
config["compute_resource"] = compute_resource
# Optional override of CPU threads
# config['cpu_cores'] = 2

# Rebuild the API with the modified configuration
api = CtrlFreeQAPI(config)

start = time.time()
_ = api.run_optimization()
elapsed = time.time() - start
print("Elapsed (s):", round(elapsed, 3))

In [ ]:
# Optional: compare CPU vs GPU on CUDA-enabled runtimes
from copy import deepcopy

if torch.cuda.is_available():
    cfg_cpu = deepcopy(config)
    cfg_cpu["compute_resource"] = "cpu"
    api_cpu = CtrlFreeQAPI(cfg_cpu)
    t0 = time.time()
    api_cpu.run_optimization()
    t1 = time.time()
    cpu_time = round(t1 - t0, 3)

    cfg_gpu = deepcopy(config)
    cfg_gpu["compute_resource"] = "gpu"
    api_gpu = CtrlFreeQAPI(cfg_gpu)
    t0 = time.time()
    api_gpu.run_optimization()
    t1 = time.time()
    gpu_time = round(t1 - t0, 3)

    print("CPU elapsed (s):", cpu_time)
    print("CUDA elapsed (s):", gpu_time)
else:
    print("CUDA not available on this runtime; CPU-only demo ran above.")